In [ ]:
!pip install konlpy

In [33]:
import gensim
from konlpy.tag import Okt
import pandas as pd
import random

In [12]:
data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv')
data.head()

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화


In [17]:
data.shape

(3870, 4)

In [4]:
model = gensim.models.Word2Vec.load('/content/drive/MyDrive/아이펠톤/ko.bin')

In [22]:
model.most_similar("돈")[0][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


'현상금'

In [71]:
def random_insertion(sen):
    noun = []
    origin = okt.morphs(sen)
    tokens = okt.pos(sen, stem=True, norm=True)
    for word in tokens:
        if word[1] in ['Noun']:
            noun.append(word[0])

    simi = []
    for n in noun:
        try:
            simi.append(model.most_similar(n)[0][0])
        except:
            continue

    ran_idx = random.randint(0,len(origin))

    for s in simi:
        origin.insert(ran_idx, s)

    return ' '.join(origin)

In [85]:
def augmen_conversation(sen):
    new_sen = []
    sen_split = sen.split('\n')
    for sen in sen_split:
        new_sen.append(random_insertion(sen)+'\n')

    new_sen = ' '.join(new_sen)
    new_sen = new_sen[:-1]
    return new_sen

In [94]:
sen = '길동 씨 이번에 이것 좀 처리해요'
random_insertion(sen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


'길동 씨 이번 에 이 것 좀 처리 입력 조금 방법 저번 정씨 신길 해요'

In [100]:
origin = list(data.non_label_sentence)

insert_conversation = []

for ori in origin:
    insert_conversation.append(augmen_conversation(ori))

insert_data = pd.DataFrame(zip(insert_conversation))
insert_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


,0
0,길동 입력 조금 방법 저번 정씨 신길 씨 이번 에 이 것 좀 처리 해요\n 이 거 ...
1,야 너 과연 에드 앞니 왜 그렇게 튀어나왔냐\n 마가 때쯤 태어날 때 부터 그랬어 ...
2,원후 야 학원 교재 사야 되는데 못가 돈독 교과서 학교 8만 원만 주면 안 돼\n ...
3,회상 첫날 독약 지난번 에드 너 저번 에 술 먹은 날 기억 해\n 아니 왜 과연\n...
4,너 또 배제 말로 각기 에드 내 말 무시 하냐\n 이 새끼 널 좆 널이 암컷 그러 ...
...,...
3865,준 하야 넌 대가리 가 왜 과연 플리트 스즈키 우승자 이렇게 크냐\n 내 머리 가 ...
3866,내 가 지금 너 아들 김길준 데리 고 있어 살리고 싶으면 계좌 에 1억만 예금 거느...
3867,나 는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살 지 살아오 젊은이 정씨 ...
3868,누구 맘대로 여기 서 장사 태수 알리 어디 하래\n 이 게 입히 어찌 그러 무슨 일...


In [103]:
data_insertion = pd.concat([data,insert_data],axis=1)
data_insertion.head()

,class,label_sentence,non_label_sentence,binary_class,0
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화,길동 입력 조금 방법 저번 정씨 신길 씨 이번 에 이 것 좀 처리 해요\n 이 거 ...
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화,야 너 과연 에드 앞니 왜 그렇게 튀어나왔냐\n 마가 때쯤 태어날 때 부터 그랬어 ...
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화,원후 야 학원 교재 사야 되는데 못가 돈독 교과서 학교 8만 원만 주면 안 돼\n ...
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화,회상 첫날 독약 지난번 에드 너 저번 에 술 먹은 날 기억 해\n 아니 왜 과연\n...
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화,너 또 배제 말로 각기 에드 내 말 무시 하냐\n 이 새끼 널 좆 널이 암컷 그러 ...


In [ ]:
data_insertion.rename(columns = {0:'inserted_data'}, inplace=True)

In [107]:
data_insertion.head()

,class,label_sentence,non_label_sentence,binary_class,inserted_data
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화,길동 입력 조금 방법 저번 정씨 신길 씨 이번 에 이 것 좀 처리 해요\n 이 거 ...
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화,야 너 과연 에드 앞니 왜 그렇게 튀어나왔냐\n 마가 때쯤 태어날 때 부터 그랬어 ...
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화,원후 야 학원 교재 사야 되는데 못가 돈독 교과서 학교 8만 원만 주면 안 돼\n ...
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화,회상 첫날 독약 지난번 에드 너 저번 에 술 먹은 날 기억 해\n 아니 왜 과연\n...
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화,너 또 배제 말로 각기 에드 내 말 무시 하냐\n 이 새끼 널 좆 널이 암컷 그러 ...


In [108]:
data_insertion.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_insertion.csv')